# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f1dd8d0c190>
├── 'a' --> tensor([[-1.3698,  0.6199,  1.1989],
│                   [-1.2224,  0.7763,  0.3758]])
└── 'x' --> <FastTreeValue 0x7f1dd8d0ca00>
    └── 'c' --> tensor([[ 0.5072,  0.3080,  0.2307, -0.4351],
                        [ 0.2413, -0.4560, -0.9637,  0.7452],
                        [-0.2740,  0.2297,  2.2098,  1.0574]])

In [4]:
t.a

tensor([[-1.3698,  0.6199,  1.1989],
        [-1.2224,  0.7763,  0.3758]])

In [5]:
%timeit t.a

79.6 ns ± 0.494 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f1dd8d0c190>
├── 'a' --> tensor([[-0.5738,  0.5888, -0.4699],
│                   [ 1.9128, -0.2771, -0.2830]])
└── 'x' --> <FastTreeValue 0x7f1dd8d0ca00>
    └── 'c' --> tensor([[ 0.5072,  0.3080,  0.2307, -0.4351],
                        [ 0.2413, -0.4560, -0.9637,  0.7452],
                        [-0.2740,  0.2297,  2.2098,  1.0574]])

In [7]:
%timeit t.a = new_value

75.6 ns ± 0.687 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.3698,  0.6199,  1.1989],
               [-1.2224,  0.7763,  0.3758]]),
    x: Batch(
           c: tensor([[ 0.5072,  0.3080,  0.2307, -0.4351],
                      [ 0.2413, -0.4560, -0.9637,  0.7452],
                      [-0.2740,  0.2297,  2.2098,  1.0574]]),
       ),
)

In [10]:
b.a

tensor([[-1.3698,  0.6199,  1.1989],
        [-1.2224,  0.7763,  0.3758]])

In [11]:
%timeit b.a

65.4 ns ± 0.411 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1392,  0.3793, -0.9528],
               [-0.3242,  0.0473,  0.3563]]),
    x: Batch(
           c: tensor([[ 0.5072,  0.3080,  0.2307, -0.4351],
                      [ 0.2413, -0.4560, -0.9637,  0.7452],
                      [-0.2740,  0.2297,  2.2098,  1.0574]]),
       ),
)

In [13]:
%timeit b.a = new_value

626 ns ± 3.03 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.02 µs ± 7.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.8 µs ± 102 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

303 µs ± 21.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

269 µs ± 5.35 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f1d345a65e0>
├── 'a' --> tensor([[[-1.3698,  0.6199,  1.1989],
│                    [-1.2224,  0.7763,  0.3758]],
│           
│                   [[-1.3698,  0.6199,  1.1989],
│                    [-1.2224,  0.7763,  0.3758]],
│           
│                   [[-1.3698,  0.6199,  1.1989],
│                    [-1.2224,  0.7763,  0.3758]],
│           
│                   [[-1.3698,  0.6199,  1.1989],
│                    [-1.2224,  0.7763,  0.3758]],
│           
│                   [[-1.3698,  0.6199,  1.1989],
│                    [-1.2224,  0.7763,  0.3758]],
│           
│                   [[-1.3698,  0.6199,  1.1989],
│                    [-1.2224,  0.7763,  0.3758]],
│           
│                   [[-1.3698,  0.6199,  1.1989],
│                    [-1.2224,  0.7763,  0.3758]],
│           
│                   [[-1.3698,  0.6199,  1.1989],
│                    [-1.2224,  0.7763,  0.3758]]])
└── 'x' --> <FastTreeValue 0x7f1d2d981130>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

45.1 µs ± 2.14 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f1d2d964280>
├── 'a' --> tensor([[-1.3698,  0.6199,  1.1989],
│                   [-1.2224,  0.7763,  0.3758],
│                   [-1.3698,  0.6199,  1.1989],
│                   [-1.2224,  0.7763,  0.3758],
│                   [-1.3698,  0.6199,  1.1989],
│                   [-1.2224,  0.7763,  0.3758],
│                   [-1.3698,  0.6199,  1.1989],
│                   [-1.2224,  0.7763,  0.3758],
│                   [-1.3698,  0.6199,  1.1989],
│                   [-1.2224,  0.7763,  0.3758],
│                   [-1.3698,  0.6199,  1.1989],
│                   [-1.2224,  0.7763,  0.3758],
│                   [-1.3698,  0.6199,  1.1989],
│                   [-1.2224,  0.7763,  0.3758],
│                   [-1.3698,  0.6199,  1.1989],
│                   [-1.2224,  0.7763,  0.3758]])
└── 'x' --> <FastTreeValue 0x7f1d2d9647c0>
    └── 'c' --> tensor([[ 0.5072,  0.3080,  0.2307, -0.4351],
                        [ 0.2413, -0.4560, -0.9637,  0.7452],
                 

In [23]:
%timeit t_cat(trees)

45.1 µs ± 4.95 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

80 µs ± 1.23 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.5072,  0.3080,  0.2307, -0.4351],
                       [ 0.2413, -0.4560, -0.9637,  0.7452],
                       [-0.2740,  0.2297,  2.2098,  1.0574]],
              
                      [[ 0.5072,  0.3080,  0.2307, -0.4351],
                       [ 0.2413, -0.4560, -0.9637,  0.7452],
                       [-0.2740,  0.2297,  2.2098,  1.0574]],
              
                      [[ 0.5072,  0.3080,  0.2307, -0.4351],
                       [ 0.2413, -0.4560, -0.9637,  0.7452],
                       [-0.2740,  0.2297,  2.2098,  1.0574]],
              
                      [[ 0.5072,  0.3080,  0.2307, -0.4351],
                       [ 0.2413, -0.4560, -0.9637,  0.7452],
                       [-0.2740,  0.2297,  2.2098,  1.0574]],
              
                      [[ 0.5072,  0.3080,  0.2307, -0.4351],
                       [ 0.2413, -0.4560, -0.9637,  0.7452],
                       [-0.2740,  0.2297,  2.2098,  1.0574]],

In [26]:
%timeit Batch.stack(batches)

104 µs ± 1.41 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.5072,  0.3080,  0.2307, -0.4351],
                      [ 0.2413, -0.4560, -0.9637,  0.7452],
                      [-0.2740,  0.2297,  2.2098,  1.0574],
                      [ 0.5072,  0.3080,  0.2307, -0.4351],
                      [ 0.2413, -0.4560, -0.9637,  0.7452],
                      [-0.2740,  0.2297,  2.2098,  1.0574],
                      [ 0.5072,  0.3080,  0.2307, -0.4351],
                      [ 0.2413, -0.4560, -0.9637,  0.7452],
                      [-0.2740,  0.2297,  2.2098,  1.0574],
                      [ 0.5072,  0.3080,  0.2307, -0.4351],
                      [ 0.2413, -0.4560, -0.9637,  0.7452],
                      [-0.2740,  0.2297,  2.2098,  1.0574],
                      [ 0.5072,  0.3080,  0.2307, -0.4351],
                      [ 0.2413, -0.4560, -0.9637,  0.7452],
                      [-0.2740,  0.2297,  2.2098,  1.0574],
                      [ 0.5072,  0.3080,  0.2307, -0.4351],
                   

In [28]:
%timeit Batch.cat(batches)

187 µs ± 1.24 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

592 µs ± 63.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
